# Top-1 NASDAQ-100 Momentum Backtest (NautilusTrader)
This notebook imports the working Strategy/Config from the module and runs the backtest.
It prints engine-native PnL and a trade log, and shows the last 5-day momentum score tables (raw and annualized), transposed with all columns visible.


In [1]:
# %pip install yfinance pandas beautifulsoup4 lxml html5lib
import sys, os
import pandas as pd
import numpy as np
import yfinance as yf

# Make module importable
sys.path.append(os.path.abspath('stock_enhanced'))
from nautilus_engine_momentum import MomentumConfig, MomentumStrategy
from nautilus_trader.backtest.engine import BacktestEngine, BacktestEngineConfig
from nautilus_trader.model.identifiers import InstrumentId, Symbol
from nautilus_trader.model.venues import Venue
from nautilus_trader.model.instruments.equity import Equity
from nautilus_trader.model.objects import Currency, Price, Quantity, Money
from nautilus_trader.model.enums import CurrencyType, OmsType, AccountType, BookType
from nautilus_trader.model.data import Bar, BarType
from nautilus_trader.model.enums import PriceType


In [2]:
def get_nasdaq100() -> list[str]:
    url = 'https://en.wikipedia.org/wiki/NASDAQ-100'
    tables = pd.read_html(url, flavor='bs4')
    comp = None
    for t in tables:
        cols = set(map(str, t.columns))
        if any('Symbol' in c or 'Ticker' in c for c in cols) and 80 <= len(t) <= 120:
            comp = t
            break
    if comp is None:
        for t in tables:
            cols = set(map(str, t.columns))
            if any('Symbol' in c or 'Ticker' in c for c in cols):
                comp = t
                break
    if comp is None:
        raise RuntimeError('Could not locate NASDAQ-100 components table')
    def _clean(s: str) -> str:
        return str(s).strip().replace('.', '-')
    col = [c for c in comp.columns if 'Symbol' in str(c) or 'Ticker' in str(c)][0]
    return comp[col].astype(str).map(_clean).tolist()


In [3]:
START_DATE='2022-01-01'
END_DATE='2025-08-20'
LOOKBACK=250
ROC=250
NUM_STOCKS=1
REB_DAYS=14
POS_SIZE=0.95
TX_COST=7.0


In [4]:
tickers = get_nasdaq100()
data = yf.download(tickers, start=START_DATE, end=END_DATE, progress=False, group_by='column')
if isinstance(data.columns, pd.MultiIndex) and 'Adj Close' in data.columns.get_level_values(0):
    close = data['Adj Close']
else:
    close = data['Close'] if isinstance(data.columns, pd.MultiIndex) else data
close = close.ffill().bfill().dropna(axis=1, how='any')
print(f'Using {len(close.columns)} NASDAQ-100 tickers')
close.tail(3)


/tmp/ipykernel_190065/3933501595.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=START_DATE, end=END_DATE, progress=False, group_by='column')


Using 101 NASDAQ-100 tickers


Ticker,AAPL,ABNB,ADBE,ADI,ADP,ADSK,AEP,AMAT,AMD,AMGN,...,TSLA,TTD,TTWO,TXN,VRSK,VRTX,WBD,WDAY,XEL,ZS
Date,,,,,,,,,,,,,,,,,,,,,
2025-08-15,231.589996,125.099998,354.850006,231.630005,301.790009,286.730011,111.989998,161.759995,177.509995,296.470001,...,330.559998,52.119999,232.179993,194.570007,267.859985,392.790009,11.85,226.089996,71.930000,274.970001
2025-08-18,230.889999,125.489998,357.239990,231.550003,302.649994,289.649994,110.699997,163.529999,176.139999,292.619995,...,335.160004,54.950001,232.550003,194.330002,265.690002,389.880005,11.82,231.850006,72.209999,277.029999
2025-08-19,230.559998,126.000000,361.029999,230.440002,305.720001,289.239990,112.660004,162.220001,166.550003,295.549988,...,329.309998,52.529999,228.360001,195.940002,271.010010,390.760010,11.56,229.789993,73.220001,274.920013


In [5]:
def to_instrument_id(t: str) -> InstrumentId: return InstrumentId(Symbol(t), Venue('SIM'))
def to_equity(iid: InstrumentId) -> Equity:
    cur = Currency('USD', 2, 840, 'US Dollar', CurrencyType.FIAT)
    px_inc = Price(0.01, 2); lot = Quantity(1, 0)
    import time; ts=int(time.time_ns())
    return Equity(iid, iid.symbol, cur, 2, px_inc, lot, ts, ts)
def bars_from_df(s: pd.Series, iid: InstrumentId):
    s=s.dropna(); last_bt=BarType.from_str(f'{iid.value}-1-DAY-LAST-EXTERNAL'); bid_bt=BarType.from_str(f'{iid.value}-1-DAY-BID-EXTERNAL'); ask_bt=BarType.from_str(f'{iid.value}-1-DAY-ASK-EXTERNAL')
    for ts, px in s.items():
        ts_ns=pd.Timestamp(ts).tz_localize('UTC').value; p=Price(float(px),2); q=Quantity(100,0)
        yield Bar(last_bt,p,p,p,p,q,ts_ns,ts_ns); yield Bar(bid_bt,p,p,p,p,q,ts_ns,ts_ns); yield Bar(ask_bt,p,p,p,p,q,ts_ns,ts_ns)


In [6]:
cfg=BacktestEngineConfig(); engine=BacktestEngine(cfg)
cur=Currency('USD',2,840,'US Dollar',CurrencyType.FIAT)
engine.add_venue(Venue('SIM'), OmsType.NETTING, AccountType.CASH, [Money(100000.0, cur)], base_currency=cur, book_type=BookType.L1_MBP, bar_execution=True, reject_stop_orders=True, support_gtd_orders=True, support_contingent_orders=True, use_position_ids=True, use_random_ids=False, use_reduce_only=False)
iids=[to_instrument_id(t) for t in close.columns]
for i in iids: engine.add_instrument(to_equity(i))
strat_cfg = MomentumConfig(instrument_ids=iids, lookback_period=LOOKBACK, roc_period=ROC, num_stocks=NUM_STOCKS, rebalance_days=REB_DAYS, position_size=POS_SIZE, transaction_cost=TX_COST, liquidate_on_last_bar=True)
strategy=MomentumStrategy(strat_cfg); engine.add_strategy(strategy)
bars=[]
for col,iid in zip(close.columns,iids): bars.extend(list(bars_from_df(close[col], iid)))
bars.sort(key=lambda b: b.ts_event); engine.add_data(bars)
engine.run(); result=engine.get_result(); print('Backtest run complete.')


2025-08-21T15:45:06.693578715Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-08-21T15:45:06.693602739Z [INFO] BACKTESTER-001.BacktestEngine:  NAUTILUS TRADER - Automated Algorithmic Trading Platform
2025-08-21T15:45:06.693605935Z [INFO] BACKTESTER-001.BacktestEngine:  by Nautech Systems Pty Ltd.
2025-08-21T15:45:06.693607318Z [INFO] BACKTESTER-001.BacktestEngine:  Copyright (C) 2015-2024. All rights reserved.
2025-08-21T15:45:06.693609692Z [INFO] BACKTESTER-001.BacktestEngine: =================================================================
2025-08-21T15:45:06.693610864Z [INFO] BACKTESTER-001.BacktestEngine: 
2025-08-21T15:45:06.693612187Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⣴⣶⡟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
2025-08-21T15:45:06.693613850Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣰⣾⣿⣿⣿⠀⢸⣿⣿⣿⣿⣶⣶⣤⣀⠀⠀⠀⠀⠀
2025-08-21T15:45:06.693615403Z [INFO] BACKTESTER-001.BacktestEngine: ⠀⠀⠀⠀⠀⠀⢀⣴⡇⢀⣾⣿⣿⣿⣿⣿⠀⣾⣿⣿⣿⣿⣿⣿⣿⠿⠓⠀⠀⠀⠀
2025-08-21T15:45

In [7]:
stats_pnls = getattr(result,'stats_pnls',{}); stats_returns=getattr(result,'stats_returns',{})
print('Engine (Nautilus) stats_pnls:'); print(stats_pnls)
print('Engine (Nautilus) stats_returns:'); print(stats_returns)
net_pnl=None
if isinstance(stats_pnls,dict):
    for k,v in stats_pnls.items():
        if isinstance(v,dict):
            for key in ('pnl_total','total','net','PnL','pnl'):
                if key in v: net_pnl=(net_pnl or 0.0)+float(v[key]); break
print('Engine Net PnL (total):', net_pnl if net_pnl is not None else 'N/A')

import pandas as _pd
log_df=_pd.DataFrame(getattr(strategy,'trade_log',[]))
if not log_df.empty:
    log_df['ts']=_pd.to_datetime(log_df['ts'],unit='ns',utc=True)
    cols=[c for c in ['side','sym','qty','price','cash','equity_mtm','ts','reason'] if c in log_df.columns]
    log_df=log_df[cols]
    try: log_df['equity_mtm_delta']=log_df['equity_mtm'].diff()
    except Exception: pass
    display(log_df)
    print('Symbols traded:', sorted(log_df['sym'].unique().tolist()))
else: print('No trades logged.')


Engine (Nautilus) stats_pnls:
{'USD': {'PnL (total)': 104417.89, 'PnL% (total)': 104.41789000000001, 'Max Winner': 73942.94, 'Avg Winner': np.float64(23461.98), 'Min Winner': np.float64(233.74), 'Min Loser': np.float64(-1026.13), 'Avg Loser': np.float64(-4297.336666666666), 'Max Loser': np.float64(-9178.38), 'Expectancy': np.float64(13052.23625), 'Win Rate': 0.625}}
Engine (Nautilus) stats_returns:
{'Returns Volatility (252 days)': np.float64(0.7734413934477805), 'Average (Return)': np.float64(0.2611559663881794), 'Average Loss (Return)': np.float64(-0.11684186181885663), 'Average Win (Return)': np.float64(0.48795466331240095), 'Sharpe Ratio (252 days)': np.float64(0.7374988845581794), 'Sortino Ratio (252 days)': np.float64(4.490069067251777), 'Profit Factor': np.float64(6.960328768538047), 'Risk Return Ratio': np.float64(0.5382650074066335)}
Engine Net PnL (total): N/A


,side,sym,qty,price,cash,equity_mtm,ts,reason,equity_mtm_delta
0,BUY,CEG.SIM,1120,84.76,100000.00,100000.00,2022-12-29 00:00:00+00:00,NaN,NaN
1,SELL,CEG.SIM,1120,82.38,5057.85,97323.45,2023-02-09 00:00:00+00:00,NaN,-2676.55
2,BUY,PDD.SIM,51,92.43,5057.85,97323.45,2023-02-09 00:00:00+00:00,NaN,0.00
3,SELL,PDD.SIM,51,72.32,92598.31,96286.63,2023-04-06 00:00:00+00:00,NaN,-1036.82
4,BUY,AXON.SIM,404,217.29,92598.31,96286.63,2023-04-06 00:00:00+00:00,NaN,0.00
5,SELL,AXON.SIM,404,194.59,8497.42,87111.78,2023-06-29 00:00:00+00:00,NaN,-9174.85
6,BUY,NVDA.SIM,196,41.09,8497.42,87111.78,2023-06-29 00:00:00+00:00,NaN,0.00
7,SELL,NVDA.SIM,196,42.30,79052.64,87343.44,2023-11-02 00:00:00+00:00,NaN,231.66
8,BUY,APP.SIM,2046,36.70,79052.64,87343.44,2023-11-02 00:00:00+00:00,NaN,0.00
9,SELL,APP.SIM,2046,72.86,12233.32,161304.88,2024-04-04 00:00:00+00:00,NaN,73961.44


Symbols traded: ['APP.SIM', 'AXON.SIM', 'CEG.SIM', 'MSTR.SIM', 'NVDA.SIM', 'PDD.SIM', 'PLTR.SIM']


In [12]:
# Last 5-day momentum score tables (raw and annualized), transposed
_pd.set_option('display.max_columns', None)
def _compute_scores_for_date_idx_annualized(close_df, lookback, roc_period, date_idx):
    out = {}
    if date_idx < max(lookback, roc_period):
        return out
    end = date_idx + 1
    for col in close_df.columns:
        try:
            cur = float(close_df.iloc[date_idx, close_df.columns.get_loc(col)])
            past = float(close_df.iloc[date_idx - roc_period, close_df.columns.get_loc(col)])
            if np.isnan(cur) or np.isnan(past) or past <= 0:
                continue
            roc = (cur - past) / past * 100.0
            roc = max(0.0, min(1.0, float(np.ceil(roc))))
            series = pd.Series(close_df[col].iloc[end - lookback:end]).replace(0, np.nan).dropna()
            if len(series) < 20:
                continue
            x = np.arange(len(series))
            slope = np.polyfit(x, np.log(series.values), 1)[0]
            ann = (1.0 + slope)**252
            score = roc * ann
            if np.isfinite(score):
                out[col] = float(score)
        except Exception:
            continue
    return out
last_idxs = list(range(max(0, len(close.index)-5), len(close.index)))

score_cols_ann = {}
for i in last_idxs:
    label = str(pd.Timestamp(close.index[i]).date())
    score_cols_ann[label] = pd.Series(_compute_scores_for_date_idx_annualized(close, LOOKBACK, ROC, i))
score_table_ann = pd.DataFrame(score_cols_ann)
if not score_table_ann.empty: score_table_ann = score_table_ann.sort_values(by=score_table_ann.columns[-1], ascending=False)
print('Annualized score (ROC x ((1+slope)^252)), last 5 days, sorted by most recent:')
display(score_table_ann.T)


Annualized score (ROC x ((1+slope)^252)), last 5 days, sorted by most recent:


,PLTR,APP,MSTR,AXON,NFLX,DASH,CRWD,AVGO,ZS,TTWO,SHOP,GILD,WBD,CEG,BKNG,CSCO,META,CCEP,ORLY,MNST,FAST,MELI,FTNT,KLAC,LRCX,TSLA,EXC,NVDA,TRI,IDXX,INTU,TMUS,MSFT,CDNS,DXCM,XEL,AEP,AMZN,CSGP,ADP,PAYX,VRSK,COST,BKR,MU,CTAS,GOOGL,HON,GOOG,ADSK,EA,PANW,MAR,SNPS,TEAM,ABNB,LIN,ADI,DDOG,INTC,ARM,KDP,CSX,TXN,PCAR,SBUX,ROST,AAPL,AMD,MRVL,CTSH,CMCSA,CDW,CHTR,ASML,AZN,BIIB,AMGN,AMAT,ADBE,CPRT,GFS,GEHC,FANG,ISRG,KHC,LULU,MCHP,MDLZ,ROP,PEP,NXPI,ODFL,ON,REGN,QCOM,PYPL,PDD,TTD,VRTX,WDAY
2025-08-13,5.388644,3.449637,2.650922,2.055645,1.950976,1.854667,1.841480,1.729760,1.723252,1.662124,1.541134,1.526207,1.444602,1.442991,1.397411,1.373986,1.319943,1.325755,1.311890,1.309408,1.291652,1.291896,1.349006,1.225510,1.222048,1.276668,1.250055,1.208914,1.246595,1.180404,1.210319,1.223641,1.178448,1.168727,1.185636,1.174116,1.152836,1.153130,1.139686,1.145571,1.152415,1.140012,1.129221,1.119922,1.082070,1.102180,1.076741,1.094868,1.074343,1.079379,1.039025,1.066146,1.063223,1.010196,1.096875,1.040435,1.016051,1.005203,0.998275,0.975912,0.975837,0.961908,0.936587,0.934469,0.943438,0.936628,0.895872,0.868085,0.831466,0.792681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.035888,0.0,0.0,0.0,0.0,1.034695,0.0,0.0,0.0,0.0,0.0,0.0,0.862617,0.0,0.0,0.0,0.933407
2025-08-14,5.373587,3.397092,2.613764,2.046625,1.946742,1.852368,1.831055,1.736090,1.726371,1.657304,1.542450,1.522843,1.440219,1.440015,1.391497,1.369981,1.322565,1.324938,1.313092,1.308748,1.293390,1.291604,1.334561,1.234900,1.232066,1.267866,1.249067,1.214779,1.243233,1.190409,1.212173,1.220674,1.182790,1.172282,1.183839,1.172020,1.154735,1.151384,1.141430,1.142421,1.147430,1.136341,1.127118,1.118314,1.088884,1.101331,1.078908,1.092549,1.076432,1.075454,1.043944,1.062420,1.058927,1.016355,1.079220,1.036484,1.016712,1.006732,0.996341,0.976442,0.974944,0.963468,0.939595,0.935396,0.941364,0.935855,0.896473,0.869163,0.840375,0.788569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.031186,0.0,0.0,0.0,0.0,1.031970,0.0,0.0,0.0,0.0,0.0,0.0,0.858176,0.0,0.0,0.0,0.000000
2025-08-15,5.354912,3.346167,2.576236,2.038068,1.942711,1.851105,1.821552,1.743749,1.731398,1.652583,1.544498,1.519236,1.437283,1.437248,1.385884,1.366546,1.325855,1.324562,1.313854,1.308732,1.295320,1.293795,1.321279,1.243302,1.241386,1.260421,1.248362,1.221492,1.240298,1.200928,1.214957,1.217874,1.187306,1.176733,1.183295,1.169665,1.156184,1.150782,1.144040,1.139342,1.142751,1.132446,1.125358,1.116632,1.096423,1.100136,1.081303,1.090289,1.078799,1.072202,1.048326,1.059472,1.055035,1.023389,1.062815,1.033189,1.017755,1.008698,0.995371,0.978519,0.974590,0.965144,0.942760,0.000000,0.939331,0.000000,0.897975,0.870409,0.849893,0.785340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.027136,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.854263,0.0,0.0,0.0,0.000000
2025-08-18,5.335735,3.295654,2.538805,2.031631,1.939446,1.850126,1.812110,1.751047,1.737190,1.648751,1.546636,1.515098,1.434937,1.434364,1.380507,1.364027,1.327720,1.323661,1.314593,1.308211,1.297685,1.295282,1.309326,1.251493,1.249902,1.253611,1.247301,1.228819,1.237531,1.211290,1.217805,1.215368,1.191443,1.181752,1.183575,1.167887,1.157381,1.150094,1.145959,1.136360,1.137792,1.128391,1.123569,1.114841,1.104830,1.098536,1.083872,1.088113,1.081295,1.068986,1.052106,1.055699,1.051138,1.030776,1.048205,1.029424,1.018743,1.010597,0.994443,0.979893,0.974560,0.966773,0.945566,0.000000,0.937364,0.934466,0.899290,0.871713,0.859496,0.782241,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.023093,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.850459,0.0,0.0,0.0,0.921842
2025-08-19,5.304083,3.239656,2.497836,2.022748,1.935783,1.848209,1.802454,1.757145,1.743125,1.644218,1.547870,1.511152,1.432715,1.431250,1.376105,1.361679,1.328934,1.323236,1.315829,1.307451,1.300032,1.297232,1.296432,1.259499,1.259236,1.247114,1.246695,1.236317,1.235243,1.221909,1.220406,1.213301,1.195344,1.186076,1.185013,1.166532,1.159205,1.149109,1.148414,1.133686,1.133114,1.1251